# Data C102 Fall 2021 Final Project - Steven
My contributions to the final project.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style(style='darkgrid')
plt.style.use('ggplot')
%matplotlib inline

## Data cleaning
Code ~stolen~ adapted from the main project notebook

In [60]:
# Load data into DataFrames
asthma = pd.read_csv('data/asthma.csv')
pm25 = pd.read_csv('data/pm25.csv')
states = pd.read_csv('data/states.csv')
fips = pd.read_csv('https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv')
state_pops = pd.read_csv('data/nst-est2019-alldata.csv')

*For hypothesis testing*:

In [68]:
# Add divisions to the asthma data
asthma_divs = asthma.merge(states, left_on='LocationAbbr', right_on='State Code').drop(columns=['State', 'State Code'])
asthma_divs.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3,Region,Division
0,2012,2012,AL,Alabama,NVSS,Asthma,Asthma mortality rate,NaN,NaN,Number,...,AST4_1,NMBR,GENDER,GENF,NaN,NaN,NaN,NaN,South,East South Central
1,2014,2014,AL,Alabama,NVSS,Asthma,Asthma mortality rate,NaN,NaN,Number,...,AST4_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN,South,East South Central
2,2015,2015,AL,Alabama,NVSS,Asthma,Asthma mortality rate,NaN,NaN,Number,...,AST4_1,NMBR,RACE,WHT,NaN,NaN,NaN,NaN,South,East South Central
3,2013,2013,AL,Alabama,NVSS,Asthma,Asthma mortality rate,NaN,"cases per 1,000,000",Crude Rate,...,AST4_1,CRDRATE,OVERALL,OVR,NaN,NaN,NaN,NaN,South,East South Central
4,2016,2016,AL,Alabama,NVSS,Asthma,Asthma mortality rate,NaN,"cases per 1,000,000",Age-adjusted Rate,...,AST4_1,AGEADJRATE,RACE,WHT,NaN,NaN,NaN,NaN,South,East South Central


In [71]:
# Query for overall age-adjusted prevalence
asthma_aap = asthma_states.query(
        'Question == "Current asthma prevalence among adults aged >= 18 years"' + 
        '& StratificationCategory1 == "Overall"' +
        '& DataValueType == "Age-adjusted Prevalence"' # Asthma prevalence is expressed as a percentage of the overall population
    )[['YearStart', 'LocationAbbr', 'LocationDesc', 'Division', 'DataValue']].rename(
        columns={'YearStart': 'year',
                 'LocationAbbr': 'state',
                 'LocationDesc': 'stname',
                 'Division': 'div',
                 'DataValue': 'aap'}
    ).reset_index().drop(columns='index')
asthma_aap.head()

,year,state,stname,div,aap
0,2019,AL,Alabama,East South Central,9.4
1,2015,AL,Alabama,East South Central,9.9
2,2017,AL,Alabama,East South Central,10.9
3,2013,AL,Alabama,East South Central,8.5
4,2011,AL,Alabama,East South Central,8.0


In [244]:
# Fill the only NA value with the average age-adjusted prevalence in NJ
NJ_aap_mean = round(asthma_aap.query('state == "NJ"').mean()['aap'], 1)
asthma_aap = asthma_aap.fillna(value={'aap': NJ_aap_mean})
asthma_aap.query('state == "NJ"')

,year,state,stname,div,aap
351,2011,NJ,New Jersey,Middle Atlantic,9.0
352,2012,NJ,New Jersey,Middle Atlantic,8.8
353,2014,NJ,New Jersey,Middle Atlantic,8.3
354,2016,NJ,New Jersey,Middle Atlantic,8.2
355,2018,NJ,New Jersey,Middle Atlantic,8.4
356,2015,NJ,New Jersey,Middle Atlantic,7.3
357,2019,NJ,New Jersey,Middle Atlantic,8.4
358,2017,NJ,New Jersey,Middle Atlantic,8.4
359,2013,NJ,New Jersey,Middle Atlantic,9.0


In [265]:
# Calculate weighted average age-adjusted prevalence for each division
state_pop_means_df = pd.DataFrame( # Calculate mean population in each state over the years 2011-2019
        list(
            map(
                lambda x: [x[0], round(np.mean(x[1:]), 0)], 
                state_pops.query('SUMLEV == 40')[['NAME'] + list(state_pops.columns[8:17])].to_numpy()
            )
        )
    ).rename(columns={1: 'pop_mean'})

asthma_aap_pop_means = asthma_aap.merge(# Merge mean population with AAP DataFrame
        state_pop_means_df, 
        left_on='stname', 
        right_on=0
    ).drop(columns=0) 

asthma_aap_pop_means['asthma_est'] = (asthma_aap_pop_means['aap'] * asthma_aap_pop_means['pop_mean'] / 100).apply( # Calculate estimated number of people with asthma
        lambda x: round(x, 0)
    ) 

asthma_div_agg = asthma_aap_pop_means.groupby(# Add up the components for calculating the weighted averages
        ['year', 'div']
    )[['pop_mean', 'asthma_est']].sum() 

asthma_aap_div = (100 * asthma_div_agg['asthma_est'] / asthma_div_agg['pop_mean']).unstack( # Calculate the weighted averages
        level=0
    ) 

asthma_aap_div

year,2011,2012,2013,2014,2015,2016,2017,2018,2019
div,,,,,,,,,
East North Central,9.290029,9.508730,9.661071,10.304176,9.568443,9.647181,9.614669,9.716669,10.129544
East South Central,8.176904,8.692163,8.064242,9.501390,9.780204,10.318155,10.068209,10.204454,9.229796
Middle Atlantic,9.394450,9.376531,9.518467,9.935867,9.464911,9.592496,9.414532,9.703424,9.695541
Mountain,8.984050,8.666093,8.699489,8.952006,9.048255,9.064734,9.574572,9.306167,9.590992
New England,10.922441,10.708664,11.167393,11.080513,10.578117,10.763634,11.549736,10.813969,10.843975
Pacific,8.765354,9.065384,9.099978,8.166292,8.220279,8.286157,8.497671,8.903215,8.431108
South Atlantic,8.486177,8.349025,8.546645,8.285746,8.227076,8.092688,8.590490,9.069895,8.240776
West North Central,8.099535,8.648817,8.798916,8.808873,8.358533,8.469037,8.571469,8.817956,9.039054
West South Central,7.721947,7.347102,7.586650,7.200149,7.882669,8.004370,7.856245,8.062787,7.601606


*For causal inference*:

In [261]:
# Add state names to the PM2.5 data
pm25_states = pm25.merge(
        fips, 
        left_on='statefips', 
        right_on=' st'
    ).drop(
        columns=['ds_pm_stdd', 'statefips', ' st']
    ).rename(
        columns={' stusps': 'state'}
    )[['year', 'state', 'stname', 'ds_pm_pred']]
pm25_states.head()

,year,state,stname,ds_pm_pred
0,2011,AL,Alabama,10.696324
1,2012,AL,Alabama,10.396601
2,2013,AL,Alabama,10.039805
3,2014,AL,Alabama,11.007064
4,2011,DE,Delaware,9.671609


In [264]:
# Merge AAP data with PM2.5 data
pm25_asthma = pm25_states.merge(
        asthma_aap,
        how='left',
        on=['year', 'stname']
    ).drop(
        columns='state_y'
    ).rename(
        columns={'state_x': 'state'}
    )[['year', 'state', 'div', 'ds_pm_pred', 'aap']]
pm25_asthma.head()

,year,state,div,ds_pm_pred,aap
0,2011,AL,East South Central,10.696324,8.0
1,2012,AL,East South Central,10.396601,8.5
2,2013,AL,East South Central,10.039805,8.5
3,2014,AL,East South Central,11.007064,9.5
4,2011,DE,South Atlantic,9.671609,10.0
...,...,...,...,...,...
191,2014,CO,Mountain,6.674491,8.3
192,2011,CT,New England,9.284398,10.1
193,2012,CT,New England,8.072119,10.1
194,2013,CT,New England,8.406929,10.0


Old code:

In [131]:
asthma_aap_list = asthma_aap.sort_values(
        ['stname', 'year']
    ).groupby(
        'stname'
    ).agg(
        {'aap': list}
    ).reset_index().rename(
        columns={'aap': 'aaps'}
    ).merge(states, left_on='stname', right_on='State').drop(
        columns=['State', 'State Code', 'Region']
    ).rename(
        columns={'Division': 'div'}
    )[['stname', 'div', 'aaps']]

asthma_aap_list

,stname,div,aaps
0,Alabama,East South Central,"[8.0, 8.5, 8.5, 9.5, 9.9, 9.8, 10.9, 10.4, 9.4]"
1,Alaska,Pacific,"[8.5, 9.1, 9.4, 8.2, 9.3, 9.0, 8.4, 9.0, 9.7]"
2,Arizona,Mountain,"[9.6, 8.7, 8.9, 9.7, 9.3, 9.3, 9.7, 10.0, 9.8]"
3,Arkansas,West South Central,"[9.5, 8.7, 8.2, 8.7, 10.0, 8.5, 9.7, 9.8, 9.3]"
4,California,Pacific,"[8.4, 8.8, 8.7, 7.7, 7.6, 7.7, 7.8, 8.5, 7.8]"
5,Colorado,Mountain,"[8.3, 8.9, 8.7, 8.3, 9.1, 8.8, 9.3, 9.2, 9.7]"
6,Connecticut,New England,"[10.1, 10.1, 10.0, 9.3, 10.7, 10.6, 10.9, 10.1..."
7,Delaware,South Atlantic,"[10.0, 10.0, 10.7, 8.7, 9.0, 8.7, 10.5, 10.3, ..."
8,District of Columbia,South Atlantic,"[10.0, 10.1, 11.9, 11.3, 10.6, 10.1, 9.1, 11.8..."
9,Florida,South Atlantic,"[7.6, 8.1, 8.2, 8.0, 7.5, 6.7, 7.5, 8.7, 7.3]"


In [117]:
NJ_means = asthma_aap_list.iloc[30, 1]
NJ_means[8] = round(np.mean(NJ_means[0:7]), 1)
asthma_aap_list

,stname,aap
0,Alabama,"[8.0, 8.5, 8.5, 9.5, 9.9, 9.8, 10.9, 10.4, 9.4]"
1,Alaska,"[8.5, 9.1, 9.4, 8.2, 9.3, 9.0, 8.4, 9.0, 9.7]"
2,Arizona,"[9.6, 8.7, 8.9, 9.7, 9.3, 9.3, 9.7, 10.0, 9.8]"
3,Arkansas,"[9.5, 8.7, 8.2, 8.7, 10.0, 8.5, 9.7, 9.8, 9.3]"
4,California,"[8.4, 8.8, 8.7, 7.7, 7.6, 7.7, 7.8, 8.5, 7.8]"
5,Colorado,"[8.3, 8.9, 8.7, 8.3, 9.1, 8.8, 9.3, 9.2, 9.7]"
6,Connecticut,"[10.1, 10.1, 10.0, 9.3, 10.7, 10.6, 10.9, 10.1..."
7,Delaware,"[10.0, 10.0, 10.7, 8.7, 9.0, 8.7, 10.5, 10.3, ..."
8,District of Columbia,"[10.0, 10.1, 11.9, 11.3, 10.6, 10.1, 9.1, 11.8..."
9,Florida,"[7.6, 8.1, 8.2, 8.0, 7.5, 6.7, 7.5, 8.7, 7.3]"


In [154]:
asthma_aap.value_counts('year')

year
2011    51
2012    51
2013    51
2014    51
2015    51
2016    51
2017    51
2018    51
2019    51
dtype: int64

In [134]:
state_pops.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 151 columns):
 #    Column                 Dtype  
---   ------                 -----  
 0    SUMLEV                 int64  
 1    REGION                 object 
 2    DIVISION               object 
 3    STATE                  int64  
 4    NAME                   object 
 5    CENSUS2010POP          int64  
 6    ESTIMATESBASE2010      int64  
 7    POPESTIMATE2010        int64  
 8    POPESTIMATE2011        int64  
 9    POPESTIMATE2012        int64  
 10   POPESTIMATE2013        int64  
 11   POPESTIMATE2014        int64  
 12   POPESTIMATE2015        int64  
 13   POPESTIMATE2016        int64  
 14   POPESTIMATE2017        int64  
 15   POPESTIMATE2018        int64  
 16   POPESTIMATE2019        int64  
 17   NPOPCHG_2010           int64  
 18   NPOPCHG_2011           int64  
 19   NPOPCHG_2012           int64  
 20   NPOPCHG_2013           int64  
 21   NPOPCHG_2014           int64  
 22   NP

In [172]:
state_pops_list = state_pops.query('SUMLEV == 40').melt(
        id_vars='NAME', 
        value_vars=state_pops.columns[8:17]
    )[['NAME', 'value']].groupby('NAME').agg(list).reset_index().rename(
        columns={'NAME': 'stname', 'value': 'pops'}
    )
state_pops_list

,stname,pops
0,Alabama,"[4799069, 4815588, 4830081, 4841799, 4852347, ..."
1,Alaska,"[722128, 730443, 737068, 736283, 737498, 74145..."
2,Arizona,"[6472643, 6554978, 6632764, 6730413, 6829676, ..."
3,Arkansas,"[2940667, 2952164, 2959400, 2967392, 2978048, ..."
4,California,"[37638369, 37948800, 38260787, 38596972, 38918..."
5,Colorado,"[5121108, 5192647, 5269035, 5350101, 5450623, ..."
6,Connecticut,"[3588283, 3594547, 3594841, 3594524, 3587122, ..."
7,Delaware,"[907381, 915179, 923576, 932487, 941252, 94892..."
8,District of Columbia,"[619800, 634924, 650581, 662328, 675400, 68581..."
9,Florida,"[19053237, 19297822, 19545621, 19845911, 20209..."


In [175]:
asthma_aap_pops = asthma_aap_list.merge(state_pops_list, on='stname')
asthma_aap_pops

,stname,div,aaps,pops
0,Alabama,East South Central,"[8.0, 8.5, 8.5, 9.5, 9.9, 9.8, 10.9, 10.4, 9.4]","[4799069, 4815588, 4830081, 4841799, 4852347, ..."
1,Alaska,Pacific,"[8.5, 9.1, 9.4, 8.2, 9.3, 9.0, 8.4, 9.0, 9.7]","[722128, 730443, 737068, 736283, 737498, 74145..."
2,Arizona,Mountain,"[9.6, 8.7, 8.9, 9.7, 9.3, 9.3, 9.7, 10.0, 9.8]","[6472643, 6554978, 6632764, 6730413, 6829676, ..."
3,Arkansas,West South Central,"[9.5, 8.7, 8.2, 8.7, 10.0, 8.5, 9.7, 9.8, 9.3]","[2940667, 2952164, 2959400, 2967392, 2978048, ..."
4,California,Pacific,"[8.4, 8.8, 8.7, 7.7, 7.6, 7.7, 7.8, 8.5, 7.8]","[37638369, 37948800, 38260787, 38596972, 38918..."
5,Colorado,Mountain,"[8.3, 8.9, 8.7, 8.3, 9.1, 8.8, 9.3, 9.2, 9.7]","[5121108, 5192647, 5269035, 5350101, 5450623, ..."
6,Connecticut,New England,"[10.1, 10.1, 10.0, 9.3, 10.7, 10.6, 10.9, 10.1...","[3588283, 3594547, 3594841, 3594524, 3587122, ..."
7,Delaware,South Atlantic,"[10.0, 10.0, 10.7, 8.7, 9.0, 8.7, 10.5, 10.3, ...","[907381, 915179, 923576, 932487, 941252, 94892..."
8,District of Columbia,South Atlantic,"[10.0, 10.1, 11.9, 11.3, 10.6, 10.1, 9.1, 11.8...","[619800, 634924, 650581, 662328, 675400, 68581..."
9,Florida,South Atlantic,"[7.6, 8.1, 8.2, 8.0, 7.5, 6.7, 7.5, 8.7, 7.3]","[19053237, 19297822, 19545621, 19845911, 20209..."


In [191]:
state_pop_means = asthma_aap_pops['pops'].apply(lambda x: int(round(np.mean(x), 0))).to_numpy()
asthma_aap_pops

,stname,div,aaps,pops,pop_mean
0,Alabama,East South Central,"[8.0, 8.5, 8.5, 9.5, 9.9, 9.8, 10.9, 10.4, 9.4]","[4799069, 4815588, 4830081, 4841799, 4852347, ...",4851973
1,Alaska,Pacific,"[8.5, 9.1, 9.4, 8.2, 9.3, 9.0, 8.4, 9.0, 9.7]","[722128, 730443, 737068, 736283, 737498, 74145...",734584
2,Arizona,Mountain,"[9.6, 8.7, 8.9, 9.7, 9.3, 9.3, 9.7, 10.0, 9.8]","[6472643, 6554978, 6632764, 6730413, 6829676, ...",6849144
3,Arkansas,West South Central,"[9.5, 8.7, 8.2, 8.7, 10.0, 8.5, 9.7, 9.8, 9.3]","[2940667, 2952164, 2959400, 2967392, 2978048, ...",2979608
4,California,Pacific,"[8.4, 8.8, 8.7, 7.7, 7.6, 7.7, 7.8, 8.5, 7.8]","[37638369, 37948800, 38260787, 38596972, 38918...",38762489
5,Colorado,Mountain,"[8.3, 8.9, 8.7, 8.3, 9.1, 8.8, 9.3, 9.2, 9.7]","[5121108, 5192647, 5269035, 5350101, 5450623, ...",5442737
6,Connecticut,New England,"[10.1, 10.1, 10.0, 9.3, 10.7, 10.6, 10.9, 10.1...","[3588283, 3594547, 3594841, 3594524, 3587122, ...",3583062
7,Delaware,South Atlantic,"[10.0, 10.0, 10.7, 8.7, 9.0, 8.7, 10.5, 10.3, ...","[907381, 915179, 923576, 932487, 941252, 94892...",940540
8,District of Columbia,South Atlantic,"[10.0, 10.1, 11.9, 11.3, 10.6, 10.1, 9.1, 11.8...","[619800, 634924, 650581, 662328, 675400, 68581...",670117
9,Florida,South Atlantic,"[7.6, 8.1, 8.2, 8.0, 7.5, 6.7, 7.5, 8.7, 7.3]","[19053237, 19297822, 19545621, 19845911, 20209...",20250086
